# How to use mml interactively

`MML` provides the `mml.interactive` module with the `mml.interactive.planning` and `mml.interactive.loading` submodules to support direct usage within jupyter notebooks. This notebook itself demonstrates the usage.

In [3]:
from pathlib import Path

import mml.interactive

# your mml.env location might differ
mml.interactive.init(env_path=Path(mml.__file__).parent / "mml.env")

## data exploration

In [6]:
# this example shows how to retrieve all installed tasks
with mml.interactive.default_file_manager() as fm:
    print(sorted(list(fm.task_index.keys())))

In [7]:
# this example loads task information
tasks = ["caltech256_object_classification", "cifar10_object_classification", "svhn"]
mml.interactive.get_task_infos(tasks)

## experiment planning

In [9]:
# this example creates some MML calls to run experiments
base_reqs = mml.interactive.DefaultRequirements()
all_cmds = list()
# create task data
all_cmds.append(
    mml.interactive.MMLJobDescription(
        prefix_req=base_reqs, mode="create", config_options={"task_list": tasks, "proj": "my_exp"}
    )
)
# preprocess tasks
all_cmds.append(
    mml.interactive.MMLJobDescription(
        prefix_req=base_reqs, mode="pp", config_options={"task_list": tasks, "proj": "my_exp"}
    )
)
# create tagged task variants
all_cmds.append(
    mml.interactive.MMLJobDescription(
        prefix_req=base_reqs,
        mode="info",
        config_options={"task_list": tasks, "proj": "my_exp", "tagging.all": "+subset?0_05+confuse?0_1"},
    )
)
# you may also loop over
for t in tasks:
    all_cmds.append(
        mml.interactive.MMLJobDescription(
            prefix_req=base_reqs,
            mode="train",
            config_options={"pivot.name": f"{t}+subset?0_05+confuse?0_1", "proj": "my_exp", "sampling.batch_size": 500},
        )
    )
# either render jobs directly
for cmd in all_cmds:
    print(cmd.render())
# or put them into a file
mml.interactive.write_out_commands(cmd_list=all_cmds, name="my_exp")

## start planned jobs from within a notebook

In [ ]:
runner = mml.interactive.EmbeddedJobRunner()
job = mml.interactive.MMLJobDescription(
    prefix_req=base_reqs,
    mode="info",
    config_options={"task_list": tasks, "proj": "my_exp", "tagging.all": "+subset?0_05+confuse?0_1"},
)
runner.run(job=job)

## experiment evaluation

In [13]:
# load experiment models
models = mml.interactive.load_project_models("test")

In [27]:
# inspect and evaluate
print(models["lapgyn4_instrument_count_miccai"][0].training_time)
print(models["lapgyn4_instrument_count_miccai"][0].performance)
print(models["lapgyn4_instrument_count_miccai"][0].metrics[-2])